# VacationPy
----

With the weather data from WeatherPy, use jupyter-gmaps and the Google Places API for planning future vacations.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('..', 'WeatherPy', 'weatherpy_cities.csv')

cities_df = pd.read_csv(csv_file)
# cities_df.count()

cities_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,0,Keningau,MY,5.3378,116.1602,74.75,98,100,1.90,2021-10-22
1,1,Mataura,NZ,-46.1927,168.8643,44.64,92,93,1.28,2021-10-22
2,2,Barguzin,RU,53.6167,109.6167,32.25,74,4,1.68,2021-10-22
3,3,Port Alfred,ZA,-33.5906,26.8910,67.71,80,98,26.62,2021-10-22
4,4,Busselton,AU,-33.6500,115.3333,54.59,72,1,9.64,2021-10-22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Convert to float
humidity = cities_df["Humidity (%)"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center = (0.0, 30.0), zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_cities_df = cities_df.loc[(cities_df["Max Temperature (F)"]>=65) & \
                                (cities_df["Max Temperature (F)"]<=85) & \
                                (cities_df["Wind Speed (mph)"]<=10) & \
                                (cities_df["Cloudiness (%)"]<=0)].dropna()

# ideal_cities_df.count()
ideal_cities_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
99,99,Cabo San Lucas,MX,22.8909,-109.9124,76.51,82,0,3.00,2021-10-22
167,167,Guerrero Negro,MX,27.9769,-114.0611,65.34,84,0,8.63,2021-10-22
192,192,Tortolì,IT,39.9256,9.6572,77.99,69,0,2.30,2021-10-22
225,225,Yerköy,TR,39.6381,34.4672,68.34,15,0,3.36,2021-10-22
252,252,Lata,PT,40.1629,-8.3327,67.35,37,0,8.34,2021-10-22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.

hotel_df = ideal_cities_df[["City","Country", "Latitude", "Longitude", \
                            "Max Temperature (F)", "Humidity (%)", \
                            "Cloudiness (%)", "Wind Speed (mph)"]].reset_index(drop = True)

hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Cabo San Lucas,MX,22.8909,-109.9124,76.51,82,0,3.00,
1,Guerrero Negro,MX,27.9769,-114.0611,65.34,84,0,8.63,
2,Tortolì,IT,39.9256,9.6572,77.99,69,0,2.30,
3,Yerköy,TR,39.6381,34.4672,68.34,15,0,3.36,
4,Lata,PT,40.1629,-8.3327,67.35,37,0,8.34,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Set parameters to search for hotels with 5000 meters.
params = {
    "keyword": "hotel", 
    "radius": 5000, 
    "types": "lodging",
    "key": g_key
}
          
for index, city in hotel_df.iterrows():
    # geocoordinates from the hotel_df
    lat = city["Latitude"]
    lng = city["Longitude"]
    params["location"] = f"{lat}, {lng}"

    # in case there's no seatch result
    try:

        hotel_response = requests.get(base_url, params = params).json()
        # print(json.dumps(response_json, indent = 4, sort_keys = True))

        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]

    except (KeyError, IndexError):
        
        hotel_df.loc[index, "Hotel Name"] = "No results"

hotel_df.head()

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Cabo San Lucas,MX,22.8909,-109.9124,76.51,82,0,3.00,Hotel Riu Palace Cabo San Lucas
1,Guerrero Negro,MX,27.9769,-114.0611,65.34,84,0,8.63,The Halfway Inn
2,Tortolì,IT,39.9256,9.6572,77.99,69,0,2.30,L'Ulivo
3,Yerköy,TR,39.6381,34.4672,68.34,15,0,3.36,Otel Murat
4,Lata,PT,40.1629,-8.3327,67.35,37,0,8.34,No results


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
max_temp = hotel_df["Max Temperature (F)"]

heat_layer = gmaps.heatmap_layer(locations, weights =[max(temp,0) for temp in max_temp],
                                 dissipating = False, max_intensity = 300, point_radius = 5)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))